# PO Data Pandas

Functions for working with PO data

In [1]:
import pandas as pd
from pathlib import Path

## Set up test data

In [18]:
od = Path('./data')
od.mkdir(parents=True, exist_ok=True)

## Set up structure

This notebook helps create two datasets from a report generated from ABB's Ellipse system. The first dataset is a report of PO lines. The second dataset is a report of PO and supplier numbers, along with a column that indicates how many lines the PO has.

In [19]:
po_column_mapping = {'Purchase_Order_Number_Combined': 'po_number', 
           'Purchase_Order_Date': 'po_date',
           'Supplier_Number': 'supplier_number',
           'Supplier_Name': 'supplier_name',
           'Stock_Code': 'stock_code',
           'UOI_Current_Quantity1': 'qty',
           'Item_Name_line': 'item_name_line',
           'Stock_Description': 'stock_description'}

po_line_column_names = list(po_column_mapping.values())
po_line_column_names.insert(1, 'line_number')

## Get last updated file

In [20]:
def get_last_file(folder):
    
    """helper function that finds the most recently updated file in a directory. 
    You use it to pull the most recent datafile dumped into a folder"""
    
    time, filepath = max((f.stat().st_mtime, f) for f in folder.iterdir())
    return filepath

In [21]:
get_last_file(od / 'Supplier Master Data')

WindowsPath('data/Supplier Master Data/SP001.xls')

## Transform Stock PO data

The build_po_lines_dataframe function creates PO lines from stock POs

In [22]:
def build_po_lines_dataframe(stock_pos, po_column_mapping, po_line_column_names):
    stock_pos = stock_pos.rename(columns=po_column_mapping)
    stock_pos['line_number'] = stock_pos.po_number.apply(lambda x: f'{x}'.split('-')[1])
    stock_pos.line_number = pd.to_numeric(stock_pos.line_number)
    stock_pos.po_number = stock_pos.po_number.apply(lambda x: f'{x}'.split('-')[0])
    stock_pos.supplier_number = stock_pos.supplier_number.apply(lambda x: f'{x}'.zfill(6))
    stock_pos.stock_description = stock_pos.stock_description.astype(str).apply(lambda x: ' '.join(x.split()))
    stock_pos.stock_code = stock_pos.stock_code.apply(lambda x: '{0:.2f}'.format(x).rstrip('0').rstrip('.'))
    stock_pos.stock_code = stock_pos.stock_code.astype('str')
    stock_pos.po_date = pd.to_datetime(stock_pos.po_date, dayfirst=True)
    stock_pos = stock_pos.sort_values(by=['po_date', 'po_number', 'line_number'], ascending=[False, True, True])
    stock_pos = stock_pos[po_line_column_names]
    return stock_pos

In [23]:
stock_po_dir = od / 'Stock PO Master Data'
stock_po_csv = get_last_file(stock_po_dir)
stock_pos = pd.read_csv(stock_po_csv)
stock_po_lines = build_po_lines_dataframe(stock_pos, po_column_mapping, po_line_column_names)
stock_po_lines.head()

,po_number,line_number,po_date,supplier_number,supplier_name,stock_code,qty,item_name_line,stock_description
0,random,1,2014-10-26,random text,random text,1307288.65,411.342288,random text,random text
99,random,100,2014-07-21,random text,random text,708370.96,574.708603,random text,random text
6,random,7,2014-07-17,random text,random text,2972024.14,512.581973,random text,random text
79,random,80,2014-06-15,random text,random text,1607064,583.478464,random text,random text
50,random,51,2014-05-01,random text,random text,1481190.73,292.804603,random text,random text


Stock PO lines are tranformed into header level POs

In [24]:
def build_po_header_dataframe(stock_po_lines):
    stock_pos_header = stock_po_lines[['po_number', 'supplier_number', 'line_number']]
    stock_pos_header = stock_pos_header.drop_duplicates(subset=['po_number'])
    stock_pos_header['line_count'] = stock_pos_header.line_number > 1
    stock_pos_header = stock_pos_header.drop(columns='line_number')
    return stock_pos_header

In [25]:
stock_po_headers = build_po_header_dataframe(stock_po_lines)
stock_po_headers.head()

,po_number,supplier_number,line_count
0,random,random text,False


Service POs are turned into header level POs and then combined with stock POs

In [32]:
def build_service_po_header_dataframe(service_pos):
    service_pos = service_pos[['Purchase Order Number', 'Supplier Number']]
    service_pos = service_pos.rename(columns={'Purchase Order Number': 'po_number', 'Supplier Number': 'supplier_number'})
    service_pos = service_pos.drop_duplicates(subset=['po_number'])
    service_pos.supplier_number = service_pos.supplier_number.apply(lambda x: f'{x}'.zfill(6))
    service_pos['line_count'] = False
    return service_pos

In [33]:
po_dir = od / 'Purchase Order Master Data'
service_po_excel = get_last_file(po_dir)
service_pos_df = pd.read_excel(service_po_excel)
service_pos = build_service_po_header_dataframe(service_pos_df)
service_pos.head()

,po_number,supplier_number,line_count
0,random text,random text,False


Combine service and stock POs

In [34]:
def combine_service_and_stock_pos(service_pos, stock_po_headers):
    all_pos = service_pos.append(stock_po_headers)
    all_pos.drop_duplicates(subset=['po_number'])
    return all_pos

In [35]:
service_pos = build_service_po_header_dataframe(service_pos_df)
all_pos = combine_service_and_stock_pos(service_pos, stock_po_headers)
all_pos.head()

,po_number,supplier_number,line_count
0,random text,random text,False
0,random,random text,False


## Build master data file

In [36]:
def build_master_vendor_dataframe(supplier_master):
    supplier_master = supplier_master[['Supplier Number','Supplier Company Name', 'ABN Number', 
                                       'Branch Code', 'Bank Account Number']]
    supplier_master = supplier_master.rename(columns={'Supplier Number': 'supplier_number', 
                                                      'Supplier Company Name': 'supplier_name', 
                                                      'ABN Number': 'abn', 'Branch Code': 'bsb', 
                                                      'Bank Account Number': 'bank_account'})
    supplier_master.supplier_number = supplier_master.supplier_number.apply(lambda x: f'{x}'.zfill(6))
    supplier_master.abn = supplier_master.abn.apply(lambda x: f'{x}'.replace(' ',''))
    supplier_master.bsb = supplier_master.bsb.apply(lambda x: f'{x}'.replace('-',''))
    return supplier_master

In [37]:
supplier_master_dir = od / 'Supplier Master Data'
supplier_master_excel = get_last_file(supplier_master_dir)
supplier_master = pd.read_excel(supplier_master_excel)
supplier_master = build_master_vendor_dataframe(supplier_master)
supplier_master.head()

,supplier_number,supplier_name,abn,bsb,bank_account
0,009122,random text,randomtext,random text,5.188683e+08
1,003912,random text,randomtext,random text,4.871164e+08
2,009589,random text,randomtext,random text,1.004975e+08
3,004960,random text,randomtext,random text,5.884282e+08
4,005666,random text,randomtext,random text,1.805070e+08


In [41]:
!jupyter nbconvert \
    --TagRemovePreprocessor.enabled=True \
    --TagRemovePreprocessor.remove_cell_tags="['build', 'test']" \
    --TemplateExporter.exclude_output=True \
    --to python "po_data_pandas_ellipse.ipynb"

first_line = """'Helper functions for working with Ellipse PO reports'

__version__ = '0.8'

"""
script_file = Path.cwd() / 'po_data_pandas_ellipse.py'
script = script_file.read_text()
script_file.write_text(first_line + script)
username = script_file.parent.parent.name
system_name = script_file.parent.name
standardised_script_name = f'pipomatic_{username}_{system_name}.py'
script_file.replace(script_file.parent / standardised_script_name)
standardised_script_name

[NbConvertApp] Converting notebook po_data_pandas_ellipse.ipynb to python
[NbConvertApp] Writing 4135 bytes to po_data_pandas_ellipse.py


'pipomatic_hudge_po_data_pandas_ellipse.py'

In [42]:
!black "pipomatic_hudge_po_data_pandas_ellipse.py"

reformatted pipomatic_hudge_po_data_pandas_ellipse.py
All done! \u2728 \U0001f370 \u2728
1 file reformatted.
